In [35]:
import os,sys, copy, pickle

In [36]:
import pandas as pd

In [2]:
from streamm import *

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
res_tag = 'local'  # Change this to remote to run the calculations remotely 
res_local = resource.Resource(res_tag)
res_local.load_json()

Reading in resource json file res_local.json 


In [5]:
peregrine = resource.Resource('peregrine')
peregrine.load_json()

Reading in resource json file res_peregrine.json 


Find sub list of particle keys to be included in et calculation

In [28]:
os.getcwd()

'/Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_esp_x20_et1'

In [29]:
os.chdir(lmp_bulk.dir['home'])

In [31]:
p3ht_n10_x20_nvt = buildingblock.Container('p3ht_n10_x20_nvt')
p3ht_n10_x20_nvt.read_cply()

In [32]:
p3ht_n10_x20_nvt.group_prop('residue','residue')

In [33]:
p3ht_n10_x20_nvt.propcompile_particles()

In [37]:
part_df =  pd.DataFrame(p3ht_n10_x20_nvt.prop_particles)

In [38]:
syC = part_df['resname'] == 'SCP2'
syS = part_df['resname'] == 'ThS'

In [40]:
sub_th = part_df[ syC | syS ]

In [41]:
struc_th = p3ht_n10_x20_nvt.getSubStructure(sub_th.index,'sub_th') 

In [42]:
struc_th.write_xyz()

Make sure the selection is correct by viewing the structure 

In [6]:
xO = 20

In [7]:
lmp_bulk = lammps.LAMMPS('p3ht_n10_esp_x%d'%(xO))

In [8]:
lmp_bulk.load_json()

Resource tag found peregrine 
Reading in resource json file res_peregrine.json 


In [9]:
os.getcwd()

'/Users/tkemper/Development/streamm_ext/streamm-tools/examples'

In [10]:
equ_et = calculation.CalculationRes('%s_et1'%(lmp_bulk.tag))

In [11]:
equ_et.set_resource(peregrine)

In [12]:
equ_et.properties['scratch'] = equ_et.dir['scratch']

In [14]:
equ_et.add_refcalc(lmp_bulk)

In [16]:
equ_et.make_dir()

Making /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_esp_x20_et1/ 


In [45]:
os.chdir(equ_et.dir['launch'])

In [49]:
file_type = 'input'
file_key = 'cply'
file_name = "p3ht_n10_x20_nvt.cply"
from_dirkey = 'home'
to_dirkey = 'launch'
equ_et.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  input cply p3ht_n10_x20_nvt.cply home launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/examples/p3ht_n10_x20_nvt.cply to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_esp_x20_et1/p3ht_n10_x20_nvt.cply 


In [22]:
file_type = 'templates'
file_key = 'run'
file_name = "streamm_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
equ_et.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates run streamm_peregrine.pbs templates launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/templates/streamm_peregrine.pbs to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_esp_x20_et1/streamm_peregrine.pbs 


In [23]:
equ_et.dir['scripts'] = '/Users/tkemper/Development/streamm_ext/streamm-tools/scripts'
print equ_et.dir['scripts']

/Users/tkemper/Development/streamm_ext/streamm-tools/scripts


In [75]:
file_type = 'scripts'
file_key = 'py_script'
file_name = "et1.py"
from_dirkey = 'scripts'
to_dirkey = 'launch'
equ_et.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  scripts py_script et1.py scripts launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/scripts/et1.py to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_esp_x20_et1/et1.py 


In [27]:
equ_et.properties['finish_str'] ='Finished'

In [43]:
list_th = [str(pkey) for pkey in sub_th.index]

In [46]:
outfile = open('list_th','wb')
outfile.write("\n".join(list_th))
outfile.close()

In [50]:
equ_et.add_file('input','list_i','list_th')

In [56]:
equ_et.properties['streamm_command'] ='python %s  %s --cply %s --list_i %s '%(equ_et.files['scripts']['py_script'],equ_et.tag,equ_et.files['input']['cply'],equ_et.files['input']['list_i'])
print equ_et.properties['streamm_command']

python et1.py  p3ht_n10_esp_x20_et1 --cply p3ht_n10_x20_nvt.cply --list_i list_th 


In [57]:
equ_et.load_str('templates','run')

In [59]:
equ_et.replacewrite_prop('run','scripts','run','%s.pbs'%(equ_et.tag))

In [61]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(equ_et.tag)
equ_et.add_file(file_type,file_key,file_name)

In [62]:
equ_et.push()

 Resource type ssh 
runnning push function in /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_esp_x20_et1 
Compressing and copying input files to scratch directory 
 file_i  p3ht_n10_x20_nvt.cply
 file_i  list_th
> compressed_file  p3ht_n10_esp_x20_et1_input.tgz
Compressing and copying templates files to scratch directory 
 file_i  streamm_peregrine.pbs
> compressed_file  p3ht_n10_esp_x20_et1_templates.tgz
Compressing and copying scripts files to scratch directory 
 file_i  p3ht_n10_esp_x20_et1.pbs
 file_i  et1.py
> compressed_file  p3ht_n10_esp_x20_et1_scripts.tgz
Copying output of reference calculations p3ht_n10_esp_x20


In [63]:
equ_et.run()

Calculation with status written 
Resource type ssh 
Executing run command ssh tkemper@peregrine.hpc.nrel.gov ' cd /scratch/tkemper/p3ht_n10_esp_x20_et1/ ; qsub p3ht_n10_esp_x20_et1.pbs '  
Executing run command ssh tkemper@peregrine.hpc.nrel.gov ' cd /scratch/tkemper/p3ht_n10_esp_x20_et1/ ; qsub et1.py '  
